In [ ]:
import os
os.environ['OPENAI_API_KEY'] = os.getenv("OPEN_AI")

In [ ]:
# Required Libraries
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from docx import Document
import datetime
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# GPT-4o model
llm = ChatOpenAI(model="gpt-4o")

# Medical Research Tool
@tool
def medical_research(query: str) -> str:
    """Searches and summarizes recent medical research related to the query."""
    return f"Simulated medical research summary for: {query}"

# Financial Research Tool
@tool
def financial_research(query: str) -> str:
    """Fetches financial insights related to the query."""
    return f"Simulated financial research summary for: {query}"

# Summarization Chain
summary_prompt = PromptTemplate.from_template(
    "Summarize the following result in 5 bullet points:\n\n{result}"
)

# Compose the summary chain using Runnable-style syntax
summary_chain = summary_prompt | llm | StrOutputParser()

@tool
def summarize_result(result: str) -> str:
    """Summarizes a result into 5 bullet points using an LLM."""
    return summary_chain.invoke({"result":result})

# Export to DOCX
@tool
def export_summary_docx(summary_text: str) -> str:
    """Exports summary to a DOCX file."""
    doc = Document()
    doc.add_heading("Summary of Results", level=1)
    for line in summary_text.split("\n"):
        doc.add_paragraph(line)
    filename = f"summary_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.docx"
    doc.save(filename)
    return f"Summary saved as {filename}"

from langgraph.prebuilt import create_react_agent

# Research Node: delegates to medical + financial
research_node = create_supervisor(
    [create_react_agent(llm, [medical_research], name="medical_researcher"),
     create_react_agent(llm, [financial_research], name="financial_researcher")],
    model=llm,
    supervisor_name="research_node",
    prompt="You are a research supervisor. Route queries to either the medical or financial researcher."
).compile(name="research_node")

# Report Node: delegates to summarizer + exporter
report_node = create_supervisor(
    [create_react_agent(llm, [summarize_result], name="summarizer"),
     create_react_agent(llm, [export_summary_docx], name="doc_exporter")],
    model=llm,
    supervisor_name="report_node",
    prompt="You are a reporting supervisor. First summarize the result, then export it to a DOCX file."
).compile(name="report_node")

# Top-level supervisor managing research and reporting
supervisor = create_supervisor(
    [research_node, report_node],
    model=llm,
    supervisor_name="main_supervisor",
    prompt="""
You are the main supervisor. First, route the query to the research_node.
Then, pass the result to the report_node for summarization and documentation.
"""
).compile(name="main_supervisor")



In [ ]:
# Input message
from langchain_core.messages import HumanMessage

response = supervisor.invoke({
    "messages": [HumanMessage(content="What are the latest trends in cancer drug development and their market impact?")]
})

print(response)



In [ ]:
HumanMessage(content='What are the latest trends in cancer drug development and their market impact?', 
             additional_kwargs={}, response_metadata={}, id='f53459a9-6867-4c9d-8dbd-b9d10d942a84'), 
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tpxY1epvVgYlX3glyQyi3ucV', 'function': {'arguments': '{}', 'name': 'transfer_to_research_node'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 108, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BojTB5sk2ZxEHsonYIuAcUlY3W0JO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='main_supervisor', id='run--eb811de1-81f7-40da-b39e-0e5f9466523e-0', tool_calls=[{'name': 'transfer_to_research_node', 'args': {}, 'id': 'call_tpxY1epvVgYlX3glyQyi3ucV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 108, 'output_tokens': 13, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='Successfully transferred to research_node', name='transfer_to_research_node', id='1d6f0071-40cd-497d-b71b-504a31f02f3b', tool_call_id='call_tpxY1epvVgYlX3glyQyi3ucV'), AIMessage(content='Here are some insights into the latest trends in cancer drug development and their market impact:\n\n1. **Personalized Medicine**: Enhanced genomic profiling and biomarker development are enabling tailored treatments, leading to better outcomes and increased market demand for such personalized therapies.\n\n2. **Targeted Therapies**: These therapies aim specifically at cancer cells, reducing side effects and promising improved patient outcomes. Their success is attracting continued investment and market growth.\n\n3. **Immunotherapy**: With approaches like checkpoint inhibitors and CAR-T cells, immunotherapy is expanding rapidly, leading to significant market growth as new drugs gain approval.\n\n4. **Combination Therapies**: By combining various treatment modalities, these therapies are showing increased efficacy, which is fostering robust market interest and investment.\n\n5. **Biologics and Biosimilars**: The rise of biologics has introduced biosimilars into the market, creating competitive dynamics and offering more affordable treatment options.\n\nIn terms of market impact:\n\n- The cost of healthcare is rising due to expensive new therapies, but the investment pipeline remains strong, attracting funding and innovation.\n  \n- Regulatory environments are evolving to expedite approvals of groundbreaking treatments, influencing market competition.\n\n- Increased competition amongst biotech and pharmaceutical firms may drive down prices and stimulate further innovation in the long run.\n\nThese trends are actively shaping oncology, with a focus on improved therapies and addressing unmet patient needs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 619, 'total_tokens': 895, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BojTQx70Q18Lji37GKvqwDElb4MIe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, name='research_node', id='run--859c0260-a3ea-4807-aff2-bcc80f5f62a2-0', usage_metadata={'input_tokens': 619, 'output_tokens': 276, 'total_tokens': 895, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Transferring back to main_supervisor', additional_kwargs={}, response_metadata={'__is_handoff_back': True}, name='research_node', id='1086a01a-3383-4498-910e-bb3056319750', tool_calls=[{'name': 'transfer_back_to_main_supervisor', 'args': {}, 'id': 'ea8f3da5-c6b6-4d81-8e12-4090983b10fb', 'type': 'tool_call'}]), ToolMessage(content='Successfully transferred back to main_supervisor', name='transfer_back_to_main_supervisor', id='3c76c91c-f6da-4b69-9410-d1b9c5f5b6c0', tool_call_id='ea8f3da5-c6b6-4d81-8e12-4090983b10fb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tvs8rIbH3E67UeO140uf0ibp', 'function': {'arguments': '{}', 'name': 'transfer_to_report_node'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 474, 'total_tokens': 486, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BojTYvMgE5ypQ7e9LquxCW3EIRgaF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='main_supervisor', id='run--69b1375c-fd36-4ce8-a583-3dc93172598d-0', tool_calls=[{'name': 'transfer_to_report_node', 'args': {}, 'id': 'call_tvs8rIbH3E67UeO140uf0ibp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 474, 'output_tokens': 12, 'total_tokens': 486, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='Successfully transferred to report_node', name='transfer_to_report_node', id='f9e5464a-7c5d-41e0-b09e-17518ca01cf2', tool_call_id='call_tvs8rIbH3E67UeO140uf0ibp'), AIMessage(content='The summary of the latest trends in cancer drug development and their market impact has been successfully exported to a DOCX file. You can download the document [here](sandbox:/mnt/data/summary_20250702_100533.docx).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 965, 'total_tokens': 1014, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BojU0efYiCZAO46yiNkGsqDjvH4ku', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, name='report_node', id='run--59f34979-1669-4062-a141-3fa7a2c107c1-0', usage_metadata={'input_tokens': 965, 'output_tokens': 49, 'total_tokens': 1014, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Transferring back to main_supervisor', additional_kwargs={}, response_metadata={'__is_handoff_back': True}, name='report_node', id='c5a69ba5-ff32-4567-82e7-045ee7d83582', tool_calls=[{'name': 'transfer_back_to_main_supervisor', 'args': {}, 'id': 'b4c0a019-4f18-4529-9a31-6c44286de6c3', 'type': 'tool_call'}]), ToolMessage(content='Successfully transferred back to main_supervisor', name='transfer_back_to_main_supervisor', id='368220da-57e5-402c-a49f-d7bb7d12bd0e', tool_call_id='b4c0a019-4f18-4529-9a31-6c44286de6c3'), AIMessage(content='The latest trends in cancer drug development focus heavily on personalized medicine, targeted therapies, immunotherapy, combination therapies, and the rise of biologics and biosimilars. These approaches are revolutionizing patient care by offering more precise, effective treatments with fewer side effects. In the market, these innovations are leading to increased investment and the evolution of regulatory practices to allow faster drug approvals.\n\nFor a detailed report, you can download the document [here](sandbox:/mnt/data/summary_20250702_100533.docx).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 611, 'total_tokens': 716, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BojU1ahrQwKQWICJ3DI7MMVl6qiCX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, name='main_supervisor', id='run--ebcc0af1-1c53-4da0-8b4e-3075d76497b8-0', usage_metadata={'input_tokens': 611, 'output_tokens': 105, 'total_tokens': 716, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}